# Fudan PRML 23Spring Assignment2


<font color='red'>*Your Name, Student ID and Date: [Name], [Student ID], [Date]*</font>

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission.*


<font color='red' size = 3> *DDL: 2023.06.11 23:59* </font>

## 1. FDUNN: your toy torch-like deep learning library (60 points)

In this assignment, you will fist implement your own torch-like deep learning library with `numpy`, named `fdunn`.

PyTorch: [Link](https://pytorch.org/)

In [23]:
# setup code, auto reload your .py file
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# numpy
import numpy as np
np.random.seed(233)

In [25]:
# matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

You have to impletement several standard deep neural modelwork modules in the `./fdunn` folder:
1.   linear/conv/pooling
2.   activation
3.   loss
4.   optim
5.   trainer

We have written most of the code for you already, and you only need to fill in the most essential parts. We have also prepared several test cases for you to check if your code works correctly.

Furthermore, you can also test the accuracy of your code by comparing its output with the output of sk-learn.

In [26]:
import sys
sys.path.append(r'C:\Users\Life_Dancer\Desktop\PRML-Spring23-FDU\assignment2\fdunn')

from fdunn import *

In [27]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

### 1. Get dataset MNIST

In [28]:
from fdunn.data.data_loader import get_dataset

dataset = 'MNIST'
trainloader, testloader = get_dataset(dataset)

### 2. FNN

In [7]:
from fdunn.modules.linear import Linear
from fdunn.modules.activation import Sigmoid
from fdunn.modules.loss import BCELoss, CrossEntropyLoss
from fdunn.optim.sgd import SGD

class FNN:
    def __init__(self, in_features, hidden_sizes, num_classes):
        self.linear1 = Linear(in_features, hidden_sizes[0])
        self.sigmoid = Sigmoid()
        self.hidden_layers = []
        for i in range(len(hidden_sizes) - 1):
            layer = Linear(hidden_sizes[i], hidden_sizes[i+1])
            self.hidden_layers.append(layer)
        self.linear_final = Linear(hidden_sizes[-1], num_classes)
        self.layers = [self.linear1, self.sigmoid] + self.hidden_layers + [self.linear_final]
    
    def forward(self, input):
        output = self.linear1.forward(input)
        output = self.sigmoid.forward(output)
        for layer in self.hidden_layers:
            output = layer.forward(output)
            output = self.sigmoid.forward(output)
        output = self.linear_final.forward(output)
        return output
    
    def backward(self, output_grad):
        output_grad = self.linear_final.backward(output_grad)
        for layer in reversed(self.hidden_layers):
            output_grad = self.sigmoid.backward(output_grad)
            output_grad = layer.backward(output_grad)
        output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.linear1.backward(output_grad)
        return output_grad

In [8]:
in_features = 784
hidden_sizes = [64, 64] 
num_classes = 10

model = FNN(in_features, hidden_sizes, num_classes)
criterion = BCELoss(model)

lr = 1e-7
optimizer = SGD(model, lr)

init: W.shape=(64, 784), b.shape=(64,)
init: W.shape=(64, 64), b.shape=(64,)
init: W.shape=(10, 64), b.shape=(10,)


In [9]:
import torch.nn.functional as F

def epoch(mode, dataloader, model, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0

    for i_batch, datum in enumerate(dataloader):
        img = datum[0]
        lab = datum[1]

        n_b = lab.shape[0]
        truth = lab

        output = model.forward(img)

        lab = F.one_hot(lab, 10)
        lab = lab.numpy()

        loss = criterion.forward(output, lab)
        predicted = np.argmax(output, 1)

        correct = (predicted == truth.numpy()).sum()

        loss_avg += loss.item()*n_b
        acc_avg += correct.item()
        num_exp += n_b

        if mode == 'train':
            criterion.backward()
            optimizer.step()
            
        # print(f"Batch: {i_batch}, Loss: {loss.item()}, Accuracy: {correct.item() / n_b}")

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [10]:
train_epochs = 20
lr_schedule = [10, 15]
decay = True

for e in range(train_epochs):

    train_loss, train_acc = epoch("train", dataloader=trainloader, model=model, optimizer=optimizer, criterion=criterion)

    test_loss, test_acc = epoch("test", dataloader=testloader, model=model, optimizer=None, criterion=criterion)

    print("Epoch: {}\tReal Acc: {}\tTest Acc: {}".format(e, train_acc, test_acc))

    if e in lr_schedule and decay:
        lr *= 0.1
        optimizer = SGD(model, lr=lr)


Epoch: 0	Real Acc: 0.11236666666666667	Test Acc: 0.1135
Epoch: 1	Real Acc: 0.11236666666666667	Test Acc: 0.1135
Epoch: 2	Real Acc: 0.1124	Test Acc: 0.1136
Epoch: 3	Real Acc: 0.11273333333333334	Test Acc: 0.114
Epoch: 4	Real Acc: 0.11518333333333333	Test Acc: 0.1156
Epoch: 5	Real Acc: 0.11788333333333334	Test Acc: 0.1172
Epoch: 6	Real Acc: 0.1195	Test Acc: 0.1182
Epoch: 7	Real Acc: 0.12048333333333333	Test Acc: 0.1186
Epoch: 8	Real Acc: 0.12041666666666667	Test Acc: 0.1197
Epoch: 9	Real Acc: 0.12138333333333333	Test Acc: 0.121
Epoch: 10	Real Acc: 0.12263333333333333	Test Acc: 0.121
Epoch: 11	Real Acc: 0.12206666666666667	Test Acc: 0.121
Epoch: 12	Real Acc: 0.1226	Test Acc: 0.1213
Epoch: 13	Real Acc: 0.12226666666666666	Test Acc: 0.1215
Epoch: 14	Real Acc: 0.12243333333333334	Test Acc: 0.1214
Epoch: 15	Real Acc: 0.12275	Test Acc: 0.1213
Epoch: 16	Real Acc: 0.12278333333333333	Test Acc: 0.1213
Epoch: 17	Real Acc: 0.1228	Test Acc: 0.1213
Epoch: 18	Real Acc: 0.12276666666666666	Test Acc: 0.

### CNN

In [43]:
from fdunn.modules.linear import Linear
from fdunn.modules.activation import Sigmoid
from fdunn.modules.loss import BCELoss, CrossEntropyLoss
from fdunn.optim.sgd import SGD
from fdunn.modules.conv import Conv2d
from fdunn.modules.pooling import MaxPool2d

class CNN:
    def __init__(self, in_channels, out_channels, kernel_size, stride_size, hidden_sizes, num_classes):
        self.conv = Conv2d(in_channels, out_channels, kernel_size)
        self.sigmoid = Sigmoid()
        self.pool = MaxPool2d(kernel_size, stride_size)
        self.hidden_layers = []
        for i in range(1, len(hidden_sizes)):
            layer = Linear(hidden_sizes[i-1], hidden_sizes[i])
            self.hidden_layers.append(layer)
        self.linear_final = Linear(hidden_sizes[-1], num_classes)
        self.layers = [self.conv, self.sigmoid] + self.hidden_layers + [self.linear_final]
    
    def forward(self, input):
        output = self.conv.forward(input)
        output = self.sigmoid.forward(output)
        output = self.pool.forward(output)
        # print('output.shape: ', output.shape)  # 将特征图展平为向量
        for layer in self.hidden_layers:
            output = layer.forward(output)
            output = self.sigmoid.forward(output)
        output = self.linear_final.forward(output)
        return output
    
    def backward(self, output_grad):
        output_grad = self.linear_final.backward(output_grad)
        for layer in reversed(self.hidden_layers):
            output_grad = layer.backward(output_grad)
            output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.pool.backward(output_grad)
        output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.conv.backward(output_grad)
        return output_grad

In [44]:
in_channels = 1
out_channels = 8
kernel_size = 5
stride_size = 2
hidden_sizes = [800]
num_classes = 10

model = CNN(in_channels, out_channels, kernel_size, stride_size, hidden_sizes, num_classes)
criterion = BCELoss(model)

lr = 1e-6
optimizer = SGD(model, lr)

init: W.shape=(10, 800), b.shape=(10,)


In [45]:
import torch.nn.functional as F

def epoch(mode, dataloader, model, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0

    for i_batch, datum in enumerate(dataloader):
        img = datum[0]
        lab = datum[1]

        n_b = lab.shape[0]
        truth = lab

        output = model.forward(img)

        lab = F.one_hot(lab, 10)
        lab = lab.numpy()

        loss = criterion.forward(output, lab)
        predicted = np.argmax(output, 1)

        correct = (predicted == truth.numpy()).sum()

        loss_avg += loss.item()*n_b
        acc_avg += correct.item()
        num_exp += n_b

        if mode == 'train':
            criterion.backward()
            optimizer.step()
            
        # print(f"Batch: {i_batch}, Loss: {loss.item()}, Accuracy: {correct.item() / n_b}")

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [46]:
train_epochs = 5
lr_schedule = [3]
decay = True

for e in range(train_epochs):

    train_loss, train_acc = epoch("train", dataloader=trainloader, model=model, optimizer=optimizer, criterion=criterion)

    print("Epoch: {}\tTrain Acc: {}".format(e, train_acc))

    if e == train_epochs-1:
        test_loss, test_acc = epoch("test", dataloader=testloader, model=model, optimizer=None, criterion=criterion)
        print("Test Acc: {}".format(test_acc))

    if e in lr_schedule and decay:
        lr *= 0.1
        optimizer = SGD(model, lr=lr)


Epoch: 0	Train Acc: 0.09771666666666666
Epoch: 1	Train Acc: 0.1009
Epoch: 2	Train Acc: 0.10346666666666667
Epoch: 3	Train Acc: 0.10463333333333333
Epoch: 4	Train Acc: 0.10518333333333334
Test Acc: 0.1025
